# Machine-learning Notebook - experimentation for calculating pick depth prediction from pick label classification prediction
- Machine_Learning_vB2_20170802
- A cleaner version of Machine_Learning_vB in the same folder
- preceeded by feature creation notebooks

In [1]:
import pandas as pd
import numpy as np
import itertools
# import matplotlib.pyplot as plt
# %matplotlib inline
import welly
from welly import Well
import lasio
import glob
from sklearn import neighbors
import pickle
import math
import dask
import dask.dataframe as dd
from dask.distributed import Client
# import pdvega
# import vega
import random
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import mean_squared_error

/Users/justingosses/anaconda/envs/MannvilleDask2/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/justingosses/anaconda/envs/MannvilleDask2/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
import multiprocessing

In [3]:
print(welly.__version__)
print(dask.__version__)
print(pd.__version__)

0.3.5
0.18.2
0.23.3


In [4]:
#### Adding this bit to silence an error that was causing the notebook to have a dead kernal
#### This is an unsafe solution but couldn't get any  of the "right solutions" to work!
#### Ended up using this = https://www.kaggle.com/c/bosch-production-line-performance/discussion/25082
#### Other solutions = https://github.com/dmlc/xgboost/issues/1715 but the solution here didn't seem to work for me?


import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [5]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.max_colwidth = 100000

In [6]:
knn_dir = "../WellsKNN/"
load_dir = "../loadLAS"
features_dir = "../createFeatures/"
machine_learning_dir = "../Machine_Learning/"

## If you open this notebook fresh and jump to a point below where a pick file is read in, you still need to load everything above! 

----------

### Load dataframes for Machine-Learning via a HDF5 file

train_X
train_y
test_X
test_y

In [7]:
h5_to_load = 'df_all_Col_preSplit_wTrainTest_ClassBalanced_PreML.h5'
train_X = pd.read_hdf(machine_learning_dir+h5_to_load, 'train_X')
train_y = pd.read_hdf(machine_learning_dir+h5_to_load, 'train_y')
test_X = pd.read_hdf(machine_learning_dir+h5_to_load, 'test_X')
test_y = pd.read_hdf(machine_learning_dir+h5_to_load, 'test_y')

In [8]:
preSplitpreBal = pd.read_hdf(machine_learning_dir+h5_to_load, 'preSplitpreBal')


In [9]:
train_X.head()

,DPHI,GR,ILD,NPHI,McMurray_Base_Qual,NN1_thickness,MM_Top_Depth_predBy_NN1thick,Quality,Quality_paleoz,DistFrom_NN1_TopDepth_Abs,BotWellDept,FromTopWell,WellThickness,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge
390,0.185,101.752,3.723,0.537,2,23.78,421.84,1,2,183.096,445.994,1.0,208.25,101.752,101.752,101.752,101.752,101.7520,101.7520,101.7520,101.7520,101.752,101.752,101.752,101.752,101.752000,101.752000,101.752,101.7520,101.752,101.752,101.752,101.752,101.752000,101.752,101.752,101.752,101.752,101.752,101.752,101.752,101.75200,101.752,101.752,101.752,3.723,3.723,3.723,3.723,3.7230,3.7230,3.7230,3.7230,3.723,3.723,3.723,3.723,3.723000,3.723000,3.7230,3.7230,3.723,3.723,3.723,3.723,3.723000,3.723,3.723,3.723,3.723,3.723,3.723,3.723,3.723000,3.723,3.723,3.723
391,0.212,100.657,2.950,0.516,2,23.78,421.84,1,2,180.596,445.994,3.5,208.25,100.349,100.657,104.476,100.657,101.5134,100.6570,100.6570,100.6570,100.349,100.657,106.802,100.657,102.304429,100.657000,100.657,100.6570,100.657,100.657,100.657,100.657,100.657000,100.657,100.657,100.657,100.657,100.657,100.657,100.657,100.65700,100.657,100.657,100.657,2.950,2.950,3.254,2.950,3.1066,2.9500,2.9500,2.9500,2.950,2.950,3.414,2.950,3.194286,2.950000,2.9500,2.9500,2.950,2.950,2.950,2.950,2.950000,2.950,2.950,2.950,2.950,2.950,2.950,2.950,2.950000,2.950,2.950,2.950
392,0.175,100.744,3.409,0.532,2,23.78,421.84,1,2,178.096,445.994,6.0,208.25,99.221,100.744,106.397,106.397,102.5820,104.5656,104.5656,102.5820,99.221,100.744,106.397,100.744,103.294000,100.744000,100.744,100.7440,99.221,100.744,106.729,100.744,103.907273,100.744,100.744,100.744,100.744,100.744,100.744,100.744,100.74400,100.744,100.744,100.744,3.299,3.299,3.558,3.493,3.4230,3.3906,3.3906,3.4230,3.299,3.409,3.632,3.409,3.449143,3.409000,3.4090,3.4090,3.299,3.409,3.632,3.409,3.478455,3.409,3.409,3.409,3.409,3.409,3.409,3.409,3.409000,3.409,3.409,3.409
393,0.265,91.018,4.864,0.489,2,23.78,421.84,1,2,175.596,445.994,8.5,208.25,67.810,91.018,102.635,102.635,88.5874,98.7966,98.7966,88.5874,58.590,91.018,102.635,105.471,85.816714,100.284000,102.356,94.8634,53.847

--------------------

In [66]:
preSplitpreBal.head()

,CALI,COND,DELT,DENS,DEPT,DEPTH,DPHI,DPHI:1,DPHI:2,DT,GR,GR:1,GR:2,IL,ILD,ILD:1,ILD:2,ILM,LITH,LLD,LLS,NPHI,PHID,PHIN,RESD,RHOB,RT,SFL,SFLU,SN,SNP,SP,UWI,SitID,McMurray_Base_HorID,McMurray_Top_HorID,McMurray_Base_DEPTH,McMurray_Top_DEPTH,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,NN1_McMurray_Top_DEPTH,NN1_McMurray_Base_DEPTH,NN1_thickness,MM_Top_Depth_predBy_NN1thick,HorID,Pick,Quality,HorID_paleoz,Pick_paleoz,Quality_paleoz,diff_TMcM_Pick_v_DEPT,diff_TPal_Pick_v_DEPT,cat_isTopMcMrNearby_known,cat_isTopPalNearby_known,DistFrom_NN1_TopDepth_Abs,NewWell,LastBitWell,TopWellDept,BotWellDept,FromTopWell,FromBotWell,WellThickness,closerToBotOrTop,closTopBotDist,rowsToEdge,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge,trainOrTest
0,167.003,NaN,NaN,NaN,149.602,NaN,0.227,NaN,NaN,NaN,102.473,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,0.460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00/10-32-080-20W4/0,112385,14000,13000,384.66,377.95,1,3,55.978836,-113.095365,389.0,414.0,25.0,359.66,13000,377.95,3,14000,384.66,1,228.348,235.058,0,0,210.058,True,False,149.602,396.102,0.00,246.50,246.5,FromTopWell,0.00,0,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,test
1,199.159,NaN,NaN,NaN,149.852,NaN,0.263,NaN,NaN,NaN,122.589,NaN,NaN,NaN,4.202,NaN,NaN,NaN,NaN,NaN,NaN,0.550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00/10-32-080-20W4/0,112385,14000,13000,384.66,377.95,1,3,55.978836,-113.095365,389.0,414.0,25.0,359.66,13000,377.95,3,14000,384.66,1,228.098,234.808,0,0,209.808,False,False,149.602,396.102,0.25,246.25,246.5,FromTopWell,0.25,1,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.58

In [10]:
seed = 123

In [11]:
# .values.ravel()
model = XGBClassifier(
    gamma=0, 
    reg_alpha=0.2, 
    max_depth=3, 
    subsample=0.8, 
    colsample_bytree= 0.8, 
    n_estimators= 300, 
    learning_rate= 0.03, 
    min_child_weight= 3,n_jobs=8)



In [12]:
model.fit(train_X,train_y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.03, max_delta_step=0,
       max_depth=3, min_child_weight=3, missing=None, n_estimators=300,
       n_jobs=8, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0.2, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

## text here

In [13]:
result_train = model.predict(train_X)
result_train

/Users/justingosses/anaconda/envs/MannvilleDask2/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([  0,   0,   0, ...,  95, 100, 100])

In [14]:
train_y_indexValues = train_y.index.values
df_result_train = pd.DataFrame(result_train, index=train_y_indexValues, columns=['TopMcMr_Pick_pred'])
df_results_train_ = pd.concat([train_y, df_result_train], axis=1)

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [16]:
accuracy = accuracy_score(df_results_train_['cat_isTopMcMrNearby_known'], df_results_train_['TopMcMr_Pick_pred'])

In [17]:
accuracy

0.7013336035670855

## text here 

In [18]:
result_test = model.predict(test_X)
result_test

/Users/justingosses/anaconda/envs/MannvilleDask2/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([ 0,  0,  0, ..., 95, 95, 95])

In [19]:
test_y_indexValues = test_y.index.values
df_result = pd.DataFrame(result_test, index=test_y_indexValues, columns=['TopMcMr_Pick_pred'])
df_results_test_ = pd.concat([test_y, df_result], axis=1)

In [20]:
accuracy = accuracy_score(df_results_test_['cat_isTopMcMrNearby_known'], df_results_test_['TopMcMr_Pick_pred'])

In [21]:
accuracy 

0.662335105335696

### test from full dataset but from training dataset again but from original dataframe so class balance is different

In [22]:
training_cols = list(train_X.columns)
training_cols

['DPHI',
 'GR',
 'ILD',
 'NPHI',
 'McMurray_Base_Qual',
 'NN1_thickness',
 'MM_Top_Depth_predBy_NN1thick',
 'Quality',
 'Quality_paleoz',
 'DistFrom_NN1_TopDepth_Abs',
 'BotWellDept',
 'FromTopWell',
 'WellThickness',
 'GR_min_5winSize_dirAroundMin',
 'GR_min_5winSize_dirAboveMin',
 'GR_min_5winSize_dirAroundMax',
 'GR_min_5winSize_dirAboveMax',
 'GR_min_5winSize_dirAroundMean',
 'GR_min_5winSize_dirAboveMean',
 'GR_min_5winSize_dirAbovenLarge',
 'GR_min_5winSize_dirAroundnLarge',
 'GR_min_7winSize_dirAroundMin',
 'GR_min_7winSize_dirAboveMin',
 'GR_min_7winSize_dirAroundMax',
 'GR_min_7winSize_dirAboveMax',
 'GR_min_7winSize_dirAroundMean',
 'GR_min_7winSize_dirAboveMean',
 'GR_min_7winSize_dirAbovenLarge',
 'GR_min_7winSize_dirAroundnLarge',
 'GR_min_11winSize_dirAroundMin',
 'GR_min_11winSize_dirAboveMin',
 'GR_min_11winSize_dirAroundMax',
 'GR_min_11winSize_dirAboveMax',
 'GR_min_11winSize_dirAroundMean',
 'GR_min_11winSize_dirAboveMean',
 'GR_min_11winSize_dirAbovenLarge',
 'GR_mi

In [23]:
training_df_origRows = preSplitpreBal[training_cols]

In [24]:
training_df_origRows.head()

,DPHI,GR,ILD,NPHI,McMurray_Base_Qual,NN1_thickness,MM_Top_Depth_predBy_NN1thick,Quality,Quality_paleoz,DistFrom_NN1_TopDepth_Abs,BotWellDept,FromTopWell,WellThickness,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge
0,0.227,102.473,0.000,0.460,1,25.0,359.66,3,1,210.058,396.102,0.00,246.5,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,0.263,122.589,4.202,0.550,1,25.0,359.66,3,1,209.808,396.102,0.25,246.5,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,4.202,4.202,4.202,4.202,4.202,4.202,4.202,4.202,4.202,4.202,4.202,4.202,4.202,4.202,4.202,4.202,4.202,4.202,4.202,4.202,4.202,4.202,4.202,4.202,4.202,4.202,4.202,4.202,4.202,4.202,4.202,4.202
2,0.252,120.196,4.643,0.537,1,25.0,359.66,3,1,209.558,396.102,0.50,246.5,120.196,120.196,120.196,120.196,120.196,120.196,120.196,120.196,120.196,120.196,120.196,120.196,120.196,120.196,120.196,120.196,120.196,120.196,120.196,120.196,120.196,120.196,120.196,120.196,120.196,120.196,120.196,120.196,120.196,120.196,120.196,120.196,4.643,4.643,4.643,4.643,4.643,4.643,4.643,4.643,4.643,4.643,4.643,4.643,4.643,4.643,4.643,4.643,4.643,4.643,4.643,4.643,4.643,4.643,4.643,4.643,4.643,4.643,4.643,4.643,4.643,4.643,4.643,4.643
3,0.244,115.975,5.280,0.513,1,25.0,359.66,3,1,209.308,396.102,0.75,246.5,115.975,115.975,115.975,115.975,115.975,115.975,115.975,115.975,115.975,115.975,115.975,115.975,115.975,115.975,115.975,115.975,115.975,115.975,115.975,115.975,115.975,115.975,115.975,115.975,115.975,115.975,115.975,115.975,115.975,115.975,115.9

In [25]:
len(training_df_origRows)

1482756

In [26]:
result_trainOrigRows = model.predict(training_df_origRows)
result_trainOrigRows

/Users/justingosses/anaconda/envs/MannvilleDask2/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([0, 0, 0, ..., 0, 0, 0])

In [27]:
len(result_trainOrigRows)

1482756

In [28]:
training_df_origRows_indexValues = training_df_origRows.index.values
df_result = pd.DataFrame(result_trainOrigRows, index=training_df_origRows_indexValues, columns=['TopMcMr_Pick_pred'])
df_results_train2_ = pd.concat([preSplitpreBal['cat_isTopMcMrNearby_known'], df_result['TopMcMr_Pick_pred']], axis=1)

In [29]:
accuracy = accuracy_score(df_results_train2_['cat_isTopMcMrNearby_known'], df_results_train2_['TopMcMr_Pick_pred'])

In [30]:
accuracy

0.8863919619950956

--------

## Go from classifiers on rows to single pick per well and error bars

In [31]:
!ls

Machine_Learning_PostSplit_vA_20180804.ipynb
Machine_Learning_vA_20170726.ipynb
Machine_Learning_vB2_20170802-Copy1.ipynb
Machine_Learning_vB2_20170802.ipynb
Machine_Learning_vC_20170802.ipynb
dask-worker-space
df_all_Col_preSplit_wTrainTest_ClassBalanced_PreML.h5
old_vB.ipynb


In [32]:
s = pd.Series([0, 0, 10, 10, 95, 10,0])

In [33]:
s.rolling(2).quantile(.5, interpolation='midpoint')

0     NaN
1     0.0
2     5.0
3    10.0
4    52.5
5    52.5
6     5.0
dtype: float64

In [34]:
s.quantile(0.5)

10.0

In [35]:
y = pd.Series([0, 0, 0, 0, 0, 0, 0, 10, 95,100,100,10,10,10,10,10,100,100,100,100,0 ,0 ,0 ,0 ])
x = pd.Series([10,11,12,13,14,15,16,17 ,18 ,19 ,20,21,22,23,24,25,26 , 27, 28,29, 30,31,32,33])

In [36]:
sum(y)

755

In [37]:
def findMiddle(x,y):
    """
    Given two arrays, x and y where y has values and x is treated like an axis
    Check that they are equal length, if not return error
    If equal length, find length
    Add up all the y values, divide by number of x values to find the mean value, then starting from the base or top,
    Sum the y values along each x value until they equal or exceed the mean value, return that x value.
    """
    x_len = len(x)
    y_len = len(y)
    print("x_len =",x_len," and y_len is ",y_len)
    if (x_len != y_len):
        return "x and y are not the same length, please check the supplied series, thank you!"
    else:
        half_y = sum(y)/2
        print("avg_y",half_y)
        temp_sum = 0
        for i in range(y_len):
            print("i=",i)
            temp_sum = y[i] + temp_sum
            print("temp_sum",temp_sum)
            if(temp_sum >= half_y):
                return x[i]
            else:
                pass
    return "error in findMiddle function"

In [38]:
test = findMiddle(x,y)
test

x_len = 24  and y_len is  24
avg_y 377.5
i= 0
temp_sum 0
i= 1
temp_sum 0
i= 2
temp_sum 0
i= 3
temp_sum 0
i= 4
temp_sum 0
i= 5
temp_sum 0
i= 6
temp_sum 0
i= 7
temp_sum 10
i= 8
temp_sum 105
i= 9
temp_sum 205
i= 10
temp_sum 305
i= 11
temp_sum 315
i= 12
temp_sum 325
i= 13
temp_sum 335
i= 14
temp_sum 345
i= 15
temp_sum 355
i= 16
temp_sum 455


26

In [39]:
def changeClassValues_partA(array_value):
    """
    Takes in an array, an array of values found in the first array, such as 0,10,50,100, and an array of ouputs of same lengh as inputs to that represents their new values
    For example, an array of values that are always 0, 60,95,100, such as [0,0,0,60,60,60,95,100,95,95,60,0,0] could be changed into values of 0s,10s,80s,and 100s. [0,0,0,10,10,10,80,100,80,80,10,0,0]
    One use case of this is to change the relative strength of each class when using it as labels or features or when calculating the weighted middle instance.
    """
    array_inputs = [0,10,60,95,100]
    array_outputs = [0,1,2,3,4]
    length_ofInputs = len([0,10,60,95,100])
    for i in range(length_ofInputs):
        if array_inputs[i] == array_value:
            return array_outputs[i]
    print("did not find a match between input array and output for given argument in changeClassValues_partA")
    return 0
        

In [40]:
test = changeClassValues_partA(95)
test

3

 apply this to a series = changeClassValues(array_value,array_inputs,array_outputs)

In [41]:
changeClassValues_partB = np.vectorize(changeClassValues_partA) #Convert python function to vector function
test2 = np.apply_along_axis(changeClassValues_partB, 0,[0, 0, 10, 10, 95, 10,0,95,100,100,60,60,0,0,0,0])
print(test2)

[0 0 1 1 3 1 0 3 4 4 2 2 0 0 0 0]


## Find median values of each prediction array + depth array of each well

In [42]:
df_results_train_.describe()

,cat_isTopMcMrNearby_known,TopMcMr_Pick_pred
count,246700.000000,246700.000000
mean,45.229712,43.371585
std,43.793369,43.146057
min,0.000000,0.000000
25%,0.000000,0.000000
50%,60.000000,60.000000
75%,95.000000,95.000000
max,100.000000,100.000000


In [43]:
df_results_train_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 246700 entries, 390 to 307647
Data columns (total 2 columns):
cat_isTopMcMrNearby_known    246700 non-null int64
TopMcMr_Pick_pred            246700 non-null int64
dtypes: int64(2)
memory usage: 5.6 MB


In [44]:
df_results_train_.head()

,cat_isTopMcMrNearby_known,TopMcMr_Pick_pred
390,0,0
391,0,0
392,0,0
393,0,0
394,0,0


In [45]:
df_results_train2_.head()

,cat_isTopMcMrNearby_known,TopMcMr_Pick_pred
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [46]:
preSplitpreBal.head()

,CALI,COND,DELT,DENS,DEPT,DEPTH,DPHI,DPHI:1,DPHI:2,DT,GR,GR:1,GR:2,IL,ILD,ILD:1,ILD:2,ILM,LITH,LLD,LLS,NPHI,PHID,PHIN,RESD,RHOB,RT,SFL,SFLU,SN,SNP,SP,UWI,SitID,McMurray_Base_HorID,McMurray_Top_HorID,McMurray_Base_DEPTH,McMurray_Top_DEPTH,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,NN1_McMurray_Top_DEPTH,NN1_McMurray_Base_DEPTH,NN1_thickness,MM_Top_Depth_predBy_NN1thick,HorID,Pick,Quality,HorID_paleoz,Pick_paleoz,Quality_paleoz,diff_TMcM_Pick_v_DEPT,diff_TPal_Pick_v_DEPT,cat_isTopMcMrNearby_known,cat_isTopPalNearby_known,DistFrom_NN1_TopDepth_Abs,NewWell,LastBitWell,TopWellDept,BotWellDept,FromTopWell,FromBotWell,WellThickness,closerToBotOrTop,closTopBotDist,rowsToEdge,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge,trainOrTest
0,167.003,NaN,NaN,NaN,149.602,NaN,0.227,NaN,NaN,NaN,102.473,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,0.460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00/10-32-080-20W4/0,112385,14000,13000,384.66,377.95,1,3,55.978836,-113.095365,389.0,414.0,25.0,359.66,13000,377.95,3,14000,384.66,1,228.348,235.058,0,0,210.058,True,False,149.602,396.102,0.00,246.50,246.5,FromTopWell,0.00,0,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,test
1,199.159,NaN,NaN,NaN,149.852,NaN,0.263,NaN,NaN,NaN,122.589,NaN,NaN,NaN,4.202,NaN,NaN,NaN,NaN,NaN,NaN,0.550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00/10-32-080-20W4/0,112385,14000,13000,384.66,377.95,1,3,55.978836,-113.095365,389.0,414.0,25.0,359.66,13000,377.95,3,14000,384.66,1,228.098,234.808,0,0,209.808,False,False,149.602,396.102,0.25,246.25,246.5,FromTopWell,0.25,1,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.58

In [47]:
len(preSplitpreBal)

1482756

In [48]:
df_results_train2_FullCol = pd.concat([preSplitpreBal,df_results_train2_], axis=1)

In [49]:
df_results_train2_FullCol.head()

,CALI,COND,DELT,DENS,DEPT,DEPTH,DPHI,DPHI:1,DPHI:2,DT,GR,GR:1,GR:2,IL,ILD,ILD:1,ILD:2,ILM,LITH,LLD,LLS,NPHI,PHID,PHIN,RESD,RHOB,RT,SFL,SFLU,SN,SNP,SP,UWI,SitID,McMurray_Base_HorID,McMurray_Top_HorID,McMurray_Base_DEPTH,McMurray_Top_DEPTH,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,NN1_McMurray_Top_DEPTH,NN1_McMurray_Base_DEPTH,NN1_thickness,MM_Top_Depth_predBy_NN1thick,HorID,Pick,Quality,HorID_paleoz,Pick_paleoz,Quality_paleoz,diff_TMcM_Pick_v_DEPT,diff_TPal_Pick_v_DEPT,cat_isTopMcMrNearby_known,cat_isTopPalNearby_known,DistFrom_NN1_TopDepth_Abs,NewWell,LastBitWell,TopWellDept,BotWellDept,FromTopWell,FromBotWell,WellThickness,closerToBotOrTop,closTopBotDist,rowsToEdge,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge,trainOrTest,cat_isTopMcMrNearby_known,TopMcMr_Pick_pred
0,167.003,NaN,NaN,NaN,149.602,NaN,0.227,NaN,NaN,NaN,102.473,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,0.460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00/10-32-080-20W4/0,112385,14000,13000,384.66,377.95,1,3,55.978836,-113.095365,389.0,414.0,25.0,359.66,13000,377.95,3,14000,384.66,1,228.348,235.058,0,0,210.058,True,False,149.602,396.102,0.00,246.50,246.5,FromTopWell,0.00,0,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,test,0,0
1,199.159,NaN,NaN,NaN,149.852,NaN,0.263,NaN,NaN,NaN,122.589,NaN,NaN,NaN,4.202,NaN,NaN,NaN,NaN,NaN,NaN,0.550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00/10-32-080-20W4/0,112385,14000,13000,384.66,377.95,1,3,55.978836,-113.095365,389.0,414.0,25.0,359.66,13000,377.95,3,14000,384.66,1,228.098,234.808,0,0,209.808,False,False,149.602,396.102,0.25,246.25,246.5,FromTopWell,0.25,1,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.58

In [50]:
len(df_results_train2_FullCol)

1482756

In [51]:
# df[['color', 'food', 'score']]
test_df = df_results_train2_FullCol[['UWI','DEPT','cat_isTopMcMrNearby_known','TopMcMr_Pick_pred']]

In [52]:
test_df

,UWI,DEPT,cat_isTopMcMrNearby_known,cat_isTopMcMrNearby_known,TopMcMr_Pick_pred
0,00/10-32-080-20W4/0,149.602,0,0,0
1,00/10-32-080-20W4/0,149.852,0,0,0
2,00/10-32-080-20W4/0,150.102,0,0,0
3,00/10-32-080-20W4/0,150.352,0,0,0
4,00/10-32-080-20W4/0,150.602,0,0,0
5,00/10-32-080-20W4/0,150.852,0,0,0
6,00/10-32-080-20W4/0,151.102,0,0,0
7,00/10-32-080-20W4/0,151.352,0,0,0
8,00/10-32-080-20W4/0,151.602,0,0,0
9,00/10-32-080-20W4/0,151.852,0,0,0


In [53]:
test_df.columns = ['UWI','DEPT','cat_isTopMcMrNearby_known','cat_isTopMcMrNearby_known_2','TopMcMr_Pick_pred']
test_df.head()

,UWI,DEPT,cat_isTopMcMrNearby_known,cat_isTopMcMrNearby_known_2,TopMcMr_Pick_pred
0,00/10-32-080-20W4/0,149.602,0,0,0
1,00/10-32-080-20W4/0,149.852,0,0,0
2,00/10-32-080-20W4/0,150.102,0,0,0
3,00/10-32-080-20W4/0,150.352,0,0,0
4,00/10-32-080-20W4/0,150.602,0,0,0


# Got too tired and stopped here

In [54]:
mean_test_df = test_df['cat_isTopMcMrNearby_known'].groupby(test_df['UWI']).mean()

In [55]:
mean_test_df

UWI
00/01-01-073-05W5/0     2.353506
00/01-03-085-15W4/0     4.208494
00/01-04-075-23W4/0     2.984166
00/01-05-085-15W4/0     3.398058
00/01-08-080-21W4/0     2.606383
00/01-09-080-13W4/0     2.660152
00/01-10-078-26W4/0     2.549428
00/01-11-082-23W4/0     2.819333
00/01-12-079-14W4/0     2.660152
00/01-21-084-17W4/0     3.154312
00/01-24-075-24W4/0     2.603613
00/01-25-083-02W4/0     2.848837
00/01-26-068-08W4/0     2.719201
00/01-27-079-23W4/0     2.549428
00/01-27-080-14W4/0     2.606383
00/01-28-078-02W4/0     2.367195
00/01-32-080-22W4/0     2.784091
00/02-01-074-05W5/0     2.353506
00/02-02-083-11W4/0     2.719201
00/02-03-071-04W5/0     4.425087
00/02-05-076-26W4/0     0.860495
00/02-06-074-04W5/0     2.399608
00/02-07-076-13W4/0     2.660152
00/02-08-078-02W5/0     2.447552
00/02-14-077-04W4/0     2.225250
00/02-17-079-03W4/0     2.353506
00/02-18-077-13W4/0     2.581665
00/02-22-071-22W4/0     3.137004
00/02-25-077-10W4/0     2.549428
00/02-26-072-13W4/0     2.660152
00/02-

In [56]:
UWIs = test_df.UWI.unique()

In [103]:
UWIs

array(['00/10-32-080-20W4/0', '00/04-15-081-09W4/0',
       '00/07-23-076-22W4/0', ..., '00/06-02-076-15W4/0',
       '00/11-18-079-03W5/0', '00/10-35-081-15W4/0'], dtype=object)

In [104]:
test_df_1well = test_df[test_df['UWI']=='00/11-18-079-03W5/0']

In [105]:
len(test_df_1well)

513

In [106]:
test_df_1well.head()

,UWI,DEPT,cat_isTopMcMrNearby_known,cat_isTopMcMrNearby_known_2,TopMcMr_Pick_pred
1481362,00/11-18-079-03W5/0,484.632,0,0,0
1481363,00/11-18-079-03W5/0,484.882,0,0,0
1481364,00/11-18-079-03W5/0,485.132,0,0,0
1481365,00/11-18-079-03W5/0,485.382,0,0,0
1481366,00/11-18-079-03W5/0,485.632,0,0,0


In [107]:
array_ = test_df_1well['TopMcMr_Pick_pred'].values

In [108]:
new_series = np.apply_along_axis(changeClassValues_partB, 0,test_df_1well['TopMcMr_Pick_pred'].values)

In [109]:
new_series

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [110]:
predicted_from_median_pick = findMiddle(test_df_1well['DEPT'].values,test_df_1well['TopMcMr_Pick_pred'].values)

x_len = 513  and y_len is  513
avg_y 4637.5
i= 0
temp_sum 0
i= 1
temp_sum 0
i= 2
temp_sum 0
i= 3
temp_sum 0
i= 4
temp_sum 0
i= 5
temp_sum 0
i= 6
temp_sum 0
i= 7
temp_sum 0
i= 8
temp_sum 0
i= 9
temp_sum 0
i= 10
temp_sum 0
i= 11
temp_sum 0
i= 12
temp_sum 0
i= 13
temp_sum 0
i= 14
temp_sum 0
i= 15
temp_sum 0
i= 16
temp_sum 0
i= 17
temp_sum 0
i= 18
temp_sum 0
i= 19
temp_sum 0
i= 20
temp_sum 0
i= 21
temp_sum 0
i= 22
temp_sum 0
i= 23
temp_sum 0
i= 24
temp_sum 0
i= 25
temp_sum 0
i= 26
temp_sum 0
i= 27
temp_sum 0
i= 28
temp_sum 0
i= 29
temp_sum 0
i= 30
temp_sum 0
i= 31
temp_sum 0
i= 32
temp_sum 0
i= 33
temp_sum 0
i= 34
temp_sum 0
i= 35
temp_sum 0
i= 36
temp_sum 0
i= 37
temp_sum 0
i= 38
temp_sum 0
i= 39
temp_sum 0
i= 40
temp_sum 0
i= 41
temp_sum 0
i= 42
temp_sum 0
i= 43
temp_sum 0
i= 44
temp_sum 0
i= 45
temp_sum 0
i= 46
temp_sum 0
i= 47
temp_sum 0
i= 48
temp_sum 0
i= 49
temp_sum 0
i= 50
temp_sum 0
i= 51
temp_sum 0
i= 52
temp_sum 0
i= 53
temp_sum 0
i= 54
temp_sum 0
i= 55
temp_sum 0
i= 56
temp_sum

In [111]:
predicted_from_median_pick

594.382

# IT looks like the depth returned is the last one, which means our function is not calculating things correctly?

In [112]:
test_df_1well

,UWI,DEPT,cat_isTopMcMrNearby_known,cat_isTopMcMrNearby_known_2,TopMcMr_Pick_pred
1481362,00/11-18-079-03W5/0,484.632,0,0,0
1481363,00/11-18-079-03W5/0,484.882,0,0,0
1481364,00/11-18-079-03W5/0,485.132,0,0,0
1481365,00/11-18-079-03W5/0,485.382,0,0,0
1481366,00/11-18-079-03W5/0,485.632,0,0,0
1481367,00/11-18-079-03W5/0,485.882,0,0,0
1481368,00/11-18-079-03W5/0,486.132,0,0,0
1481369,00/11-18-079-03W5/0,486.382,0,0,0
1481370,00/11-18-079-03W5/0,486.632,0,0,0
1481371,00/11-18-079-03W5/0,486.882,0,0,0


In [113]:
UWI_of_well = test_df_1well['UWI'][0:1].values[0]
UWI_of_well

'00/11-18-079-03W5/0'

In [114]:
preSplitpreBal['McMurray_Top_DEPTH']

0          377.95
1          377.95
2          377.95
3          377.95
4          377.95
5          377.95
6          377.95
7          377.95
8          377.95
9          377.95
10         377.95
11         377.95
12         377.95
13         377.95
14         377.95
15         377.95
16         377.95
17         377.95
18         377.95
19         377.95
20         377.95
21         377.95
22         377.95
23         377.95
24         377.95
25         377.95
26         377.95
27         377.95
28         377.95
29         377.95
30         377.95
31         377.95
32         377.95
33         377.95
34         377.95
35         377.95
36         377.95
37         377.95
38         377.95
39         377.95
40         377.95
41         377.95
42         377.95
43         377.95
44         377.95
45         377.95
46         377.95
47         377.95
48         377.95
49         377.95
50         377.95
51         377.95
52         377.95
53         377.95
54         377.95
55        

In [115]:
real_pick_depth = preSplitpreBal.loc[preSplitpreBal['UWI'] == test_df_1well['UWI'][0:1].values[0]]['McMurray_Top_DEPTH'][0:1].values[0]

In [116]:
real_pick_depth

578.21

In [117]:
pred_diff_real_TopMcMr = real_pick_depth - predicted_from_median_pick
pred_diff_real_TopMcMr

-16.171999999999912

In [ ]:
def calcPredVsRealTopDif(pred_df,real_df):
    """
    Function takes in:
        A dataframe with predictions and dataframe with all the original columns included known pick depth
    Function returns:
        A column for predicted dataframe with calculated single prediction depth pick based on the median row technique
        A column for predicted dataframe with calculated single prediction depth pick based on the highest number of 100s or 95s in a row if no 100s present.
        A new dataframe that is just one row per well and includes as col UWIs, known picks, predicted picks, and difference
        A score of mean abosolute error across all wells in the given dataframe.
    """
    